In [ ]:
from utils.utils import load_ood_data, compute_kde
from utils.yaml import read_yaml_file
import torch
from scipy import io as scio
import os
import numpy as np
from sklearn import metrics
import pandas as pd

In [ ]:
## test
dataset = 'Houston'
model = 'TLRSR'
root = os.path.join('anomaly_detection', 'code-for-PCA-TLRSR')
files = os.listdir(os.path.join(root,f'{dataset}_result'))
ood_list = [0]
anomaly_score = scio.loadmat(os.path.join(root, f'{dataset}_result', files[0]))['anomaly_score']

# 1 Organize result for TLRSR

In [ ]:
dataset = 'PaviaU'
model = 'TLRSR'
root = os.path.join('anomaly_detection', 'code-for-PCA-TLRSR')
files = os.listdir(os.path.join(root,f'{dataset}_result'))
# ood_list = [0]
ood_list = [0,1,2,3,4,5,6,7,8]

In [ ]:
results = []

In [ ]:
for ood in ood_list:
    config = read_yaml_file(path='', directory='configs', file_name=f'ood_config_{dataset}')
    config['data']['ood_left_out_classes'] = [ood,]
    data, num_classes = load_ood_data(config['data'])
    for file in files:
        result = {}
        result['model'] = 'TLRSR'
        result['dataset'] = dataset
        result['ood'] = ood
        name = file.strip('.mat').split('_')
        result['lambda_1'] = name[-2]
        result['lambda_2'] = name[-1]
        # anomaly_score
        # print(file)
        anomaly_score = scio.loadmat(os.path.join(root, f'{dataset}_result', file))['anomaly_score']
        anomaly_score_1D = np.float32(anomaly_score.reshape(-1,))
        # get the roc and pr
        anomaly_score_1D_selected = anomaly_score_1D[data.labeled_indices]
        mask = data.ood_test_mask |data.id_test_mask
        corrects = data.ood_mask
        fpr, tpr, _ = metrics.roc_curve(corrects[mask], anomaly_score_1D_selected[mask])
        roc = metrics.auc(fpr, tpr)
        prec, rec, _ = metrics.precision_recall_curve(corrects[mask], anomaly_score_1D_selected[mask])
        pr = metrics.auc(rec, prec)
        result['test_id_oa_mean'] = 'n.a.'
        result['test_id_oa_std'] = 'n.a'
        result['test_roc_mean'] = roc
        result['test_roc_std'] = 'n.a'
        result['test_pr_mean'] = pr
        result['test_pr_std'] = 'n.a.'
        results.append(result)

In [ ]:
df = pd.DataFrame(results)
df.to_csv(f'./results/TLRST_{model}_{dataset}.csv')

# 2. organize the result for RGAE

In [ ]:
dataset = 'Houston'
model = 'RGAE'
root = os.path.join('anomaly_detection', 'Hyperspectral-anomaly-detection-with-RGAE-main')
files = os.listdir(os.path.join(root,f'{dataset}_result'))
# ood_list = [0]
ood_list = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14]
results = []

In [ ]:
for ood in ood_list:
    config = read_yaml_file(path='', directory='configs', file_name=f'ood_config_{dataset}')
    config['data']['ood_left_out_classes'] = [ood,]
    data, num_classes = load_ood_data(config['data'])
    for file in files:
        result = {}
        result['model'] = 'RGAE'
        result['dataset'] = dataset
        result['ood'] = ood
        name = file.strip('.mat').split('_')
        result['lambda'] = name[-3]
        result['S'] = name[-2]
        result['n_hid'] = name[-1]
        # anomaly_score
        # print(file)
        anomaly_score = scio.loadmat(os.path.join(root, f'{dataset}_result', file))['y']
        anomaly_score_1D = np.float32(anomaly_score.reshape(-1,))
        # get the roc and pr
        anomaly_score_1D_selected = anomaly_score_1D[data.labeled_indices]
        mask = data.ood_test_mask |data.id_test_mask
        corrects = data.ood_mask
        fpr, tpr, _ = metrics.roc_curve(corrects[mask], anomaly_score_1D_selected[mask])
        roc = metrics.auc(fpr, tpr)
        prec, rec, _ = metrics.precision_recall_curve(corrects[mask], anomaly_score_1D_selected[mask])
        pr = metrics.auc(rec, prec)
        result['test_id_oa_mean'] = 'n.a.'
        result['test_id_oa_std'] = 'n.a'
        result['test_roc_mean'] = roc
        result['test_roc_std'] = 'n.a'
        result['test_pr_mean'] = pr
        result['test_pr_std'] = 'n.a.'
        results.append(result)

In [ ]:
df = pd.DataFrame(results)
df.to_csv(f'./results/ad_{model}_{dataset}.csv')

## plot

In [ ]:
# plot roc and pr figure
import plotly.express as px
from sklearn.metrics import auc
def plot_roc(fpr, tpr):

    fig = px.area(
        x=fpr, y=tpr,
        title=f'ROC Curve (AUC={auc(fpr, tpr):.4f})',
        labels=dict(x='False Positive Rate', y='True Positive Rate'),
        width=700, height=500
    )
    fig.add_shape(
        type='line', line=dict(dash='dash'),
        x0=0, x1=1, y0=0, y1=1
    )

    fig.update_yaxes(scaleanchor="x", scaleratio=1)
    fig.update_xaxes(constrain='domain')
    return fig

def plot_pr(fpr, tpr, precision, recall):
    fig = px.area(
        x=recall, y=precision,
        title=f'Precision-Recall Curve (AUC={auc(fpr, tpr):.4f})',
        labels=dict(x='Recall', y='Precision'),
        width=700, height=500
    )
    fig.add_shape(
        type='line', line=dict(dash='dash'),
        x0=0, x1=1, y0=1, y1=0
    )
    fig.update_yaxes(scaleanchor="x", scaleratio=1)
    fig.update_xaxes(constrain='domain')

    return fig
fig_roc = plot_roc(fpr, tpr)
fig_pr = plot_pr(fpr, tpr, prec, rec)

In [ ]:
fig_roc.show()

In [ ]:
fig_pr.show()